# Solving Linear Systems using Gaussian Elimination

In [54]:
clear, format compact, format short g, 

## Reduced row echelon form

`rref` reduces an augmented matrix to a (reduced) row echelon form

In [55]:
% unique solution
A = [1 2 3;
     4 5 6;
     7 8 10];
b = [2 5 9]';
rref([A b])

ans =
     1     0     0     1
     0     1     0    -1
     0     0     1     1


In [56]:
% no solution
B = A;
B(3,3) = 9;
rref([B b])

ans =
     1     0    -1     0
     0     1     2     0
     0     0     0     1


In [57]:
% infinitely many solution
c = b;
c(3) = 8;
rref([B c])

ans =
     1     0    -1     0
     0     1     2     1
     0     0     0     0


## Gaussian elimination with no pivoting

Here is our toy problem.

In [62]:
A = [2 2 1;
    -4 6 1;
    5 -5 3]
b = [6 -8 4]'

A =
     2     2     1
    -4     6     1
     5    -5     3
b =
     6
    -8
     4


Let's save copies of these for later use.

In [63]:
A_original = A; b_original = b;

### Elementary way

**Step 1. Row reduction** 

Since row operations affect both the matrix $A$ and the right-hand side $\mathbf{b}$, let's concatenate them first.

In [64]:
Ab = [A b]; % augmented matrix

Let's zero out all elements in the first column under $(1,1)$ element:
$$
R_{i} \longrightarrow R_{i} + \alpha_i R_{1}
$$
where $\alpha_i = -A_{i,1}/A_{1,1}$ for i = 2 and 3.

In [67]:
alpha = -Ab(2,1)/Ab(1,1);
Ab(2,:) = Ab(2,:) + alpha*Ab(1,:);
alpha = -Ab(3,1)/Ab(1,1);
Ab(3,:) = Ab(3,:) + alpha*Ab(1,:);

Now we zero out elements in the second column that are under (2,2) entry:
$$
R_3 \longrightarrow R_3 + \alpha R_2
$$
where $\alpha = -A_{3,2}/A_{2,2}$. Note here that $A$ has been modified and is different from the original.

In [68]:
alpha = -Ab(3,2)/Ab(2,2);
Ab(3,:) = Ab(3,:) + alpha*Ab(2,:);

**Step 2: Backward substitution**

The augmented matrix is now reduced to an echelon form which we now separate into an upper triangular matrix $U$ and a new right-hand side $\beta$.

In [71]:
U = Ab(:,1:3)
beta = Ab(:,end)

U =
            2            2            1
            0           10            3
            0            0          3.5
beta =
     6
     4
    -7


We finally solve for $\mathbf{x}$ using backward substitution.

In [86]:
x = zeros(3,1);
x(3) = beta(3)/U(3,3);
x(2) = (beta(2) - U(2,3)*x(3))/U(2,2);
x(1) = (beta(1) - U(1,2)*x(2) - U(1,3)*x(3))/U(1,1);
x

x =
     3
     1
    -2


Let's check our work. We will calculate the difference $A\mathbf{x} - \mathbf{b}$ which is called the *residual*; we want this to be 0, but due to round-off errors it rarely happens. Instead, we expect to see a tiny residual.

In [87]:
resid = A_original*x - b_original

resid =
     0
     0
     0


### Matrix way

In this section, we will carry out row operations in terms of matrix (left) multiplication. The matrices involved are called the *Gaussian transformations*.

In [91]:
A = A_original;
I = eye(size(A));
% first column
G1 = I;
G1(2,1) = -A(2,1)/A(1,1);
G1(3,1) = -A(3,1)/A(1,1);
A = G1*A;
% second column
G2 = I;
G2(3,2) = -A(3,2)/A(2,2);
A = G2*A
beta = G2*G1*b

A =
            2            2            1
            0           10            3
            0            0          3.5
beta =
     6
     4
    -7


Now the same backward substitution routine will solve this upper triangular system and yield the same result.

### Generalization: using for-loops

Now we write a general-purpose code for any given inputs $A \in \mathbb{R}^n$ and $\mathbf{b} \in \mathbb{R}^n$. 

In [132]:
%% script m-file: Gaussian elimination without pivoting
% input: A, b 
% output: x (satisfying A*x = b)
n = 10;
A = rand(n);
b = rand(n,1);
% n = size(A, 1);
Ab = [A b];
for j = 1:n-1
    for i = j+1:n
        alpha = -Ab(i,j)/Ab(j,j);
        Ab(i,j:end) = Ab(i,j:end) + alpha*Ab(j,j:end);
    end
end
U = Ab(:,1:end-1);
beta = Ab(:,end);
x = zeros(n,1);
x(n) = beta(n)/U(n,n);
for i = n-1:-1:1
    x(i) = (beta(i) - U(i,i+1:n)*x(i+1:n))/U(i,i);
end

norm(A*x - b)

ans =
   3.3198e-15


We finally solve for $\mathbf{x}$ using backward substitution whose general formula is given by
$$
x_n = \beta_n/U_{n,n},
$$
and
$$
x_i = (\beta_i - \sum_{j=i+1}^{n} U_{i,j} x_j)/U_{i,i}
$$
for $i = n-1, n-2, \ldots, 2, 1$.

In [15]:
% lower triangular system
L = [4 0 0 0; 
     3 -1 0 0;
     -1 0 3 0;
     1 -1 -1 2];
b = [8; 5; 0; 1];
x = L\b;
residual = L*x - b;
norm(residual)

ans =
   1.1102e-16


In [16]:
% forward elimination
[m, n] = size(L);
x = zeros(m, 1);
x(1) = b(1)/L(1,1);
for i = 2:n
    x(i) = (b(i) - L(i,1:i-1)*x(1:i-1))/L(i,i);
end
norm(L*x - b)

ans =
   1.1102e-16


In [52]:
% test
n = 10;
L = tril(randi(n, n));
b = rand(n, 1);
x_matlab = L\b;

x = zeros(n, 1);
x(1) = b(1)/L(1, 1);
for i = 2:n
    x(i) = (b(i) - L(i, 1:i-1)*x(1:i-1))/L(i,i);
end

res_matlab = norm(L*x_matlab - b);
res = norm(L*x - b);
fprintf('-- Forward elimination routine\n')
fprintf('size of matrix: n = %d\n', n)
fprintf('residual (matlab backslash): %8.4e\n', res_matlab)
fprintf('residual (custom routine)  : %8.4e\n', res)



-- Forward elimination routine
size of matrix: n = 10
residual (matlab backslash): 4.2999e-16
residual (custom routine)  : 9.9301e-16
